In [1]:
import sys 
sys.path.append("..")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from WideDeep.data import get_criteo_data_dict  # 针对性修改
from Tools.utils.data import DataGenerator
from Tools.models.ranking import WideDeep, DeepFM, DCN
# from Tools.models.ranking import WideDeep, DeepFM, DCN, DCNv2, FiBiNet, EDCN, DeepFFM, FatDeepFFM
from Tools.trainers import CTRTrainer

dataset_path = '../data/criteo.csv'
model_name='widedeep'
epoch = 10
learning_rate = 1e-3
batch_size=2048
weight_decay=1e-3
save_dir='./'
seed=2023
# device='cpu'
device = torch.device("cuda")

/data0/ygq/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 数据处理
torch.manual_seed(seed)
dense_feas, sparse_feas, x, y = get_criteo_data_dict(dataset_path)

dg = DataGenerator(x, y)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(split_ratio=[0.7, 0.1], batch_size=batch_size)

data load finished


100%|████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 39.44it/s]

the samples of train : val : test are  70000 : 10000 : 20000


In [3]:
model = WideDeep(wide_features=sparse_feas, deep_features=dense_feas, mlp_params={"dims": [256, 128], "dropout": 0.2, "activation": "relu"})
# model = DeepFM(deep_features=dense_feas, fm_features=sparse_feas, mlp_params={"dims": [256, 128], "dropout": 0.2, "activation": "relu"})
# model = DCN(features=dense_feas + sparse_feas, n_cross_layers=3, mlp_params={"dims": [256, 128]})

In [4]:
ctr_trainer = CTRTrainer(model, optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, n_epoch=epoch, earlystop_patience=10, device=device, model_path=save_dir)
#scheduler_fn=torch.optim.lr_scheduler.StepLR,scheduler_params={"step_size": 2,"gamma": 0.8},
ctr_trainer.fit(train_dataloader, val_dataloader)
auc = ctr_trainer.evaluate(ctr_trainer.model, test_dataloader)
print(f'test auc: {auc}')

epoch: 0


validation: 100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.04it/s]


epoch: 0 validation: auc: 0.7231835139586751
epoch: 1


validation: 100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.12it/s]


epoch: 1 validation: auc: 0.7406176682783154
epoch: 2


...

epoch: 8 validation: auc: 0.7775881875423749
epoch: 9


validation: 100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.77it/s]


epoch: 9 validation: auc: 0.7748450938079351


validation: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.87it/s]

test auc: 0.7655021653502974
